In [3]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from azure.ai.ml.dsl import pipeline
from azure.ai.ml import load_component
from azure.ai.ml import Input

In [4]:
fs = Input(
    #type="uri_folder",
    path="wasbs://demo@data4mldemo6150520719.blob.core.windows.net/mnist-fashion/",
    )

In [8]:
print(fs)

{'type': 'uri_folder', 'path': 'wasbs://demo@data4mldemo6150520719.blob.core.windows.net/mnist-fashion/'}


In [5]:
fashion_ds = Input(
    # why there is no type here. Is this input a uri_folder, or just a string?
    path="wasbs://demo@data4mldemo6150520719.blob.core.windows.net/mnist-fashion/"
)

In [16]:
print(Input)

<class 'azure.ai.ml.entities._inputs_outputs.Input'>


In [6]:
import os
from pathlib import Path
from mldesigner import command_component, Input, Output

# Warning: here the mldesginer Input replaced previous defined Input
# <class 'azure.ai.ml.entities._inputs_outputs.Input'>
# after mldesigner the input class is
# <class 'mldesigner._input_output.Input'> <class 'mldesigner._input_output.Output'>

[2022-09-25 07:18:58,873][mldesigner][WARNING] - Mldesigner requires azure-ai-ml >= 0.1.0b7 package to be fully functional.It's highly recommended to install the latest azure-ai-ml package.


In [8]:
# Error: tutorial sample code and pictures are inconsistent. https://learn.microsoft.com/en-us/azure/machine-learning/how-to-create-component-pipeline-python
# Error: including the image using dsl.component and the sample using component
# Error: including the image using environemtn = conda_env, the sample using direct env construct code
@command_component(
    name="prep_data", # Question: here the name prep_data and the function below prep_data_component, should it be the same?
    version = "1",
    description= "description of the prep_data component",
    display_name="display name of prep_data component",
    environment= dict(  
        conda_file= "./conda.yaml",
        image="mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04",
    ),
)
def my_component(
    input_data: Input(type="uri_folder"),
    training_data: Output(type="uri_folder"),
    test_data: Output(type="uri_folder"),
):
    convert(
            os.path.join(input_data, "train-images-idx3-ubyte"),
            os.path.join(input_data, "train-labels-idx1-ubyte"),
            os.path.join(training_data, "mnist_train.csv"),
            60000,
        )
    convert(
        os.path.join(input_data, "t10k-images-idx3-ubyte"),
        os.path.join(input_data, "t10k-labels-idx1-ubyte"),
        os.path.join(test_data, "mnist_test.csv"),
        10000,
    )


    def convert(imgf, labelf, outf, n):
        f = open(imgf, "rb")
        l = open(labelf, "rb")
        o = open(outf, "w")

        f.read(16)
        l.read(8)
        images = []

        for i in range(n):
            image = [ord(l.read(1))]
            for j in range(28 * 28):
                image.append(ord(f.read(1)))
            images.append(image)

        for image in images:
            o.write(",".join(str(pix) for pix in image) + "\n")
        f.close()
        o.close()
        l.close()    


AttributeError: type object 'CommandComponent' has no attribute '_load_from_dict'